In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
#Counting how many genes/onts are in each file
for f_path in sorted(glob.glob('../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/*.csv')):
    df = pd.read_csv(f_path)
    print(f_path)
    print(df['gene'].nunique(),'genes')
    print(df['annotation'].nunique(),'onts')
    

../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/vz_Liver_mouse1_central_gene_ont.csv
385 genes
53 onts
../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/vz_Liver_mouse1_peripheral_gene_ont.csv
385 genes
53 onts
../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/vz_Liver_mouse1_punctate_gene_ont.csv
385 genes
53 onts
../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/vz_Liver_mouse1_radial_gene_ont.csv
384 genes
53 onts
../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/vz_Liver_mouse2_central_gene_ont.csv
385 genes
54 onts
../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/vz_Liver_mouse2_peripheral_gene_ont.csv
385 genes
54 onts
../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/vz_Liver_mouse2_punctate_gene_ont.csv
385 genes
54 onts
../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/vz_Liver_mouse2_radial_gene_ont.csv
385 genes
53 onts


In [3]:
metrics = ['peripheral','central','radial','punctate']
for metric in metrics:
    print(metric)
    f_paths = glob.glob('../../nf_pipeline/outputs/vz_Liver_20220601_direct/gene_ont/*{}*.csv'.format(metric))
    df = pd.concat((pd.read_csv(p) for p in f_paths))
    

    
    df_all_slices = df.groupby(['annotation','gene']).filter(lambda g: g['experiment'].nunique() >= 2)
    filt_df = df_all_slices[df_all_slices['med_gene_spots'].ge(5)]
    
    sig_df = filt_df[
        filt_df['bh_p'].le(0.05)
    ]

    num_high_expression_groups = filt_df.groupby(['annotation','gene']).ngroups
    num_sig_high_expression_groups = sig_df.groupby(['annotation','gene']).ngroups
    
    
    print('Num sig ann/gene groups >=5 med_gene_spots',num_sig_high_expression_groups)
    print('Num tot ann/gene groups >=5 med_gene_spots',num_high_expression_groups)
    print('Frac sig ann/gene groups >=5 med_gene_spots',num_sig_high_expression_groups/num_high_expression_groups)
    
    print('Num sig genes',sig_df['gene'].nunique())
    print('Num sig ontologies',sig_df['annotation'].nunique())
    
    num_groups = sig_df.groupby(['annotation','gene']).ngroups
    print('Num sig ann/gene groups',num_groups)
    
    fraction_positive = sig_df['med_score'].gt(0).sum()/len(sig_df)
    print('Fraction sig ann/gene groups with a positive score {:.3f}'.format(
        fraction_positive
    ))
    
    num_groups_same_effect = sig_df.groupby(['annotation','gene'])['med_score'].apply(lambda v: all(v>0) or all(v<0)).sum()

    print('Number sig ann/gene groups with same direction of effect {}'.format(
        num_groups_same_effect
    ))
    
    print('Fraction sig ann/gene groups with same direction of effect {:.3f}'.format(
        num_groups_same_effect/num_groups
    ))
    
    print('There are Blank- MERFISH genes in the significant group',sig_df['gene'].str.contains('Blank-').any())
    print('')


peripheral
Num sig ann/gene groups >=5 med_gene_spots 1399
Num tot ann/gene groups >=5 med_gene_spots 1642
Frac sig ann/gene groups >=5 med_gene_spots 0.8520097442143727
Num sig genes 112
Num sig ontologies 52
Num sig ann/gene groups 1399
Fraction sig ann/gene groups with a positive score 0.621
Number sig ann/gene groups with same direction of effect 1296
Fraction sig ann/gene groups with same direction of effect 0.926
There are Blank- MERFISH genes in the significant group False

central
Num sig ann/gene groups >=5 med_gene_spots 1398
Num tot ann/gene groups >=5 med_gene_spots 1642
Frac sig ann/gene groups >=5 med_gene_spots 0.8514007308160779
Num sig genes 112
Num sig ontologies 52
Num sig ann/gene groups 1398
Fraction sig ann/gene groups with a positive score 0.370
Number sig ann/gene groups with same direction of effect 1301
Fraction sig ann/gene groups with same direction of effect 0.931
There are Blank- MERFISH genes in the significant group False

radial
Num sig ann/gene groups 

In [6]:
134/385

0.34805194805194806